In [1]:
## Workout Analytics -01, 02-
#### 01. Aquisition with Crawling
#### 02. Transformation with Cleansing, Denoising and Normalization(using Exercise-Dictionary)

In [2]:
import sys
import ssl
from datetime import datetime
from urllib.request import Request, urlopen
from functools import reduce
from itertools import count


resfetch = []
try:
    ssl._create_default_https_context = ssl._create_unverified_context

    for idx in count(start=1):
        l = len(resfetch)
        resfetch[l:] = list(filter(
            lambda s: s.startswith('#'),
            reduce(
                lambda param, func: func(param),
                [lambda raw: [] if raw is None else raw.splitlines(), lambda strs: [f(s) for f in [str.strip] for s in strs]],
                urlopen(Request('https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/datasets/Week0%02d.md' % idx)).read().decode('utf-8', errors='replace')
            )
        ))

        if len(resfetch) == l:
            break
except Exception as err:
    (lambda e: print(f'{e} : {datetime.now()}', file=sys.stderr))(err)

print(resfetch)

['## 2021/06/24', '#### 1.Flat Bench Press\\[Machine, Smith\\]: 40kg 12reps 5sets', '#### 2.Seated Chest Press\\[Machine\\]: 20kg 12reps 5sets', '#### 3.Lat Pulldown\\[Machine\\]: 20kg 12reps 5sets', '#### 4.Seated Low Row\\[Cable\\]: 15kg 12reps 3sets', '#### 5.Assisted Pullup\\[Machine\\]: -50kg 12reps 3sets', '#### 6.Shoulder Press\\[Machine\\]: 5kg 12reps 5sets', '## 2021/06/25', '#### 1.Low Pulley\\[Machine\\]: 20kg 12reps 5sets', '#### 2.Assisted Pullup\\[Machine\\]: -50kg 12reps 5sets', '#### 3.Shoulder Press\\[Machine\\]: 15kg 12reps 5sets(5th failed)', '#### 4.Leg Extension\\[Machine\\]: 20kg 12reps 3sets, 25kg 12reps 2sets, 30kg 10reps 2sets', '#### 5.Lying Leg Curls\\[Machine\\]: 20kg 12reps 5sets(5th failed)', '#### 6.Side Lateral Raise\\[Dumbbell\\]: 4kg 12reps 5sets', '## 2021/06/28', '#### 1.Flat Bench Press\\[Machine, Smith\\]: 40kg 12reps 5sets', '#### 2.Chest Fly\\[Machine, Pec Deck\\]: 20kg 12reps 5sets', '#### 3.Shoulder Press\\[Dumbbell\\]: 5kg 12reps 5sets', '####

HTTP Error 404: Not Found : 2022-09-06 11:32:53.944497


In [3]:
from pathlib import Path
import os
import pickle


# Load Exercise Dictionary
dictfile = os.path.join(Path(os.getcwd()).parent, 'datasets', 'exercise-dict.pkl')
with open(dictfile, 'rb') as file:
    exercisedict = pickle.load(file)

print(f'Loding Exercise Dictionary Pickle[{dictfile}]')

Loding Exercise Dictionary Pickle[/Users/kickscar/DoWork/PycharmProjects/HELLCHANG-PRACTICES/datasets/exercise-dict.pkl]


In [4]:
import re
from datetime import datetime

# Tranformation
restransform = []
exs_dict = exercisedict['exercises']
eqptdials_dict = exercisedict['equipment-dialects']

for res in resfetch:
    if res.startswith('####'):
        # 1. split exercise name & record
        exname, exrecord = (s.strip() for s in res.replace('####', '').strip().split(':'))

        # 2. normalize exercise name with equipment
        exname = re.sub(r'[\d+\\.]', '', exname).strip()
        pttrmatch = re.compile(r'([a-zA-Z\s\',]+)\s*\[([a-zA-Z\s\',]+)\]').match(exname)
        exname, equipment = (exname, None) if pttrmatch is None else pttrmatch.groups()

        # 2-1. name
        exname = exname.strip().title()
        for exkey in exs_dict:
            exdials = exs_dict[exkey]['dialects']
            if exname in exdials:
                exname = exkey
                break

        # 2-2. equipment
        if equipment:
            equipment = equipment.strip().title()
            for eqptdials_key in eqptdials_dict:
                if equipment in eqptdials_dict[eqptdials_key]:
                    equipment = eqptdials_key
                    break

            exname = f'{exname}[{equipment}]'

        restransform[len(restransform):] = [(date, exname, *re.split(r'\s+', exrecord.strip()))]
    elif res.startswith('##'):
        date = datetime.strptime(res.replace('##', '').strip(), '%Y/%m/%d')

print(restransform)

[(datetime.datetime(2021, 6, 24, 0, 0), 'Flat Bench Press[Smith Machine]', '40kg', '12reps', '5sets'), (datetime.datetime(2021, 6, 24, 0, 0), 'Seated Chest Press[Machine]', '20kg', '12reps', '5sets'), (datetime.datetime(2021, 6, 24, 0, 0), 'Lat Pulldown[Machine]', '20kg', '12reps', '5sets'), (datetime.datetime(2021, 6, 24, 0, 0), 'Seated Low Row[Cable]', '15kg', '12reps', '3sets'), (datetime.datetime(2021, 6, 24, 0, 0), 'Assisted Pull-Up[Machine]', '-50kg', '12reps', '3sets'), (datetime.datetime(2021, 6, 24, 0, 0), 'Shoulder Press[Machine]', '5kg', '12reps', '5sets'), (datetime.datetime(2021, 6, 25, 0, 0), 'Low Pulley[Machine]', '20kg', '12reps', '5sets'), (datetime.datetime(2021, 6, 25, 0, 0), 'Assisted Pull-Up[Machine]', '-50kg', '12reps', '5sets'), (datetime.datetime(2021, 6, 25, 0, 0), 'Shoulder Press[Machine]', '15kg', '12reps', '5sets(5th', 'failed)'), (datetime.datetime(2021, 6, 25, 0, 0), 'Leg Extension[Machine]', '20kg', '12reps', '3sets,', '25kg', '12reps', '2sets,', '30kg', 